### Libraries

In [1]:
from pymongo import MongoClient

### Connecting with MongoCompas

In [2]:
client = MongoClient()

In [4]:
companies = client.companies.companies # to conect with the companies collection
client.companies.offices # to generate a new collection with the separated offices inside the companies ddbb

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies'), 'offices')

### Convert the companies data in the correct format

As some companies have several offices the first step was separate the different offices

In [7]:
res = companies.aggregate([{"$unwind":"$offices"},{"$project":{"_id":0}}])
offices.insert_many(res)

In [16]:
res = companies.aggregate([
    {"$unwind":"$offices"},
    {"$match":{"offices.latitude":{"$ne":None}, "offices.longitude":{"$ne":None}}},
    {"$project":{"_id":0}}])

offices.drop()  # Delelting previously inserted data with nulls.
offices.insert_many(res)

#### GeoJSON

In [17]:
for comp in res:
    geojson = {
        "type":"Point",
        "coordinates":[comp["offices"]["longitude"], comp["offices"]["latitude"]]
    } 
    offices.update_one(comp, {"$set":{"geojson":geojson}})  # We update all of the elements with the new value.

In [18]:
offices.create_index([("geojson", "2dsphere")])

'geojson_2dsphere'

### Geospatial Queries